In [1]:
import pandas as pd
import numpy as np
import nltk
import ast #this model is used to convert string to list

In [2]:
movies = pd.read_csv('../data/tmdb_5000_movies.csv')
credits = pd.read_csv('../data/tmdb_5000_credits.csv')

In [3]:
movies = movies.merge(credits, on='title')

In [4]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [5]:
movies.dropna(inplace=True)

In [6]:
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [7]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [8]:
def convert3(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i['name'])
            counter += 1
    return l

In [9]:
movies['cast'] = movies['cast'].apply(convert3)

In [10]:
def fetch_director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

In [11]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [12]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [13]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])

In [14]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [15]:
new_df = movies[['movie_id','title','tags']]

In [16]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [17]:
def stem(text):
    y = []
    
    for i in text:
        y.append(ps.stem(i))
    
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv =  CountVectorizer(max_features = 5000, stop_words='english')

In [21]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [24]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [25]:
import pickle

In [26]:
pickle.dump(new_df, open('movies.pkl','wb'))

In [27]:
new_df.to_csv('Cleaned_DF.csv')

In [28]:
pickle.dump(similarity, open('similarity.pkl','wb'))